In [19]:
import numpy as np
import pandas as pd
from numpy_financial import irr

# Feasibility Analysis of a wind pumped hydroelectric storage system on the Island of Brava

# Load data
VentoCurvaPotencia = np.loadtxt('VentoCurvaPotencia.txt')
#Pd = pd.read_excel('Brava_consumo.xlsx', skiprows=2, usecols=range(8, 28))  # Assuming the data starts from row 3
# Carregue o arquivo Excel
xl = pd.ExcelFile('Brava_consumo.xlsx')
#Pd = pd.read_excel('Brava_consumo.xlsx', sheet_name='Carga2010-2037')
# Leia os dados da aba "Carga2010-2037"
Pd = pd.read_excel(xl, sheet_name='Carga2010-2037', dtype=float,nrows=8762)
# Wind Turbine Groups: [Power, Quantity, Corresponding column, Reservoir volume]

Cen = np.zeros((7, 4))  # inicializacao da matriz de cenarios
Cen[0, :] = [150, 2, 2, 1500]
Cen[1, :] = [300, 1, 3, 1500]

Cen[2, :] = [150, 3, 2, 2250]
Cen[3, :] = [225, 2, 4, 2250]

Cen[4, :] = [225, 3, 4, 3400]
Cen[5, :] = [300, 2, 3, 3000]
Cen[6, :] = [300, 3, 3, 4500]

# 1a-potencia do aerogerador;
# 2a-quantidade de aerogeradores; 
# 3a-especifica a coluna onde vai pegar a potencia do aerogerador correspondente; 2(150kW), 3(225kW) e 4(300kW)
# 4a-volume do reservatório em m3

Res = np.zeros((Cen.shape[0], 14))  # inicializacao da matriz dos resultados dos n cenários 
Res2 = np.zeros((Cen.shape[0], 8))  # inicializacao da matriz dos resultados dos n cenários


#for k in range(Cen.shape[0]):
for k in range(len(Cen)):
    
    ano = 20  # anos do projecto
    RS = Cen[k, 3]  # capacidade do reservatorio em m3
    ERmax = RS / 3600 * 9.8 * 110  # capacidade do reservatorio em kWh
    h = 8760  # quantidade de horas de simulacao
    Er = np.zeros((h, ano))  # inicializacao do vector-energia do reservatorio
    Erj = ERmax / 2  # energia inicial no reservatorio
    Pt = np.zeros((h, ano))
    Pb = np.zeros((h, ano))
    Erej = np.zeros((h, ano))
    Pge = np.zeros((h, ano))
    Pwr = np.zeros((h, ano))
    nb = 0.86  # eficiencia media das bombas
    nt = 0.86  # eficiencia media das turbinas hidraulicas
    nw = 0.97  # eficiencia electrica da conversao da potencia eolica
    Te = Cen[k, 0]  # potencia unitaria do aerogerador
    ne = Cen[k, 1]  # numero de aerogeradores
    PDmin = 256 * 0.5  # o grupo eletrogéneo não pode funcionar abaixo de 50% da carga. 256 é a potencia em kW do grupo menor
     
    Pwt = ne * nw * VentoCurvaPotencia[:, int(Cen[k, 2]) - 1]
    #Pwt = ne * nw * VentoCurvaPotencia[0:h, Cen[k, 2]]  # pega os 8760 horarios dos dados de potencia aerogeradores na 2, 3 ou 4a coluna e os coloc num unico vector;
    # a 2 coluna tem a potencia do Aerogerador 150 NTK, a 3a coluna sao as potencias horarias do NTK 300 e a 4a as potencias do aerogerador ACSA 27/225 
    Ptmax = np.mean(Pwt)  # potencia nominal da turbina hidraulica

#-------------------------------------------------
'''
# Assuming ano and h are defined, and Pwt, Pd, nb, ERmax, Erj are initialized appropriately
for i in range(1, ano + 1):  # for each year, from 2018(1) to 2037(20)
    for j in range(1, h +1):  # for each hour in the year
#for i in range(ano):  # for each year, from 2018(1) to 2037(20)
    #for j in range(h):
        
        # --------------  WIND PRODUCTION AND PUMPING  ----------------------
        #if Pwt[j] > Pd.iloc[j, i] * 0.35:
            #Pwr[j, i] = Pd.iloc[j, i] * 0.35
        if Pwt[j - 1] > Pd[j - 1, i - 1] * 0.35:  # if the park produces more than the limit supported by the grid
            
            Pwr[j - 1, i - 1] = Pd[j - 1, i - 1] * 0.35  # injects wind at the maximum value Eir into the grid
            
            # utilize the excess wind to pump to the reservoir...
            if (Pwt[j - 1] - Pwr[j - 1, i - 1]) * nb < (ERmax - Erj):  # check if the reservoir has capacity to absorb the pumping of water and maximum power is not exceeded
                Pb[j - 1, i - 1] = Pwt[j - 1] - Pwr[j - 1, i - 1]  # register the energy spent on pumping, considering the efficiency of the pump
                Erj += (Pwt[j - 1] - Pwr[j - 1, i - 1]) * nb  # pump all the energy to the reservoir
                
            else:  # or pump part of the excess and reject what the reservoir cannot absorb
                Pb[j - 1, i - 1] = (ERmax - Erj) / nb  # energy spent on pumping
                Erj = ERmax  # fill the reservoir
                Erej[j - 1, i - 1] = (Pwt[j - 1] - Pwr[j - 1, i - 1]) - (ERmax - Erj) / nb  # energy is dissipated or rejected, in the amount that the reservoir cannot receive
                
        else:  # if the park produces below the limit supported by the grid, all wind power is injected into the grid
            Pwr[j - 1, i - 1] = Pwt[j - 1]
            # Er[j - 1, i - 1] = Erj  # reservoir maintains its level

'''

for i in range(ano):
        for j in range(h):
            if Pwt[j] > Pd.iloc[j, i] * 0.35:
                Pwr[j, i] = Pd.iloc[j, i] * 0.35

                if (Pwt[j] - Pwr[j, i]) * nb < (ERmax - Erj):
                    Pb[j, i] = Pwt[j] - Pwr[j, i]
                    Erj += (Pwt[j] - Pwr[j, i]) * nb
                else:
                    Pb[j, i] = (ERmax - Erj) / nb
                    Erj = ERmax
                    Erej[j, i] = (Pwt[j] - Pwr[j, i]) - (ERmax - Erj) / nb
            else:
                Pwr[j, i] = Pwt[j]            

#------------

#--------------- energia de compensacao: hidraulica ou diesel --------


if (Pd.iloc[j, i] - Pwr[j][i]) <= Erj * nt:  # Check if reservoir has enough hydraulic energy to complement wind energy
            
    if (Pd.iloc[j][i] - Pwr[j][i]) <= Ptmax:  # Check if the hydroturbine has enough power
                Pt[j][i] = Pd.iloc[j][i] - Pwr[j][i]  # Hydraulic turbine injects remaining electricity into the grid
                Erj -= Pt[j][i] / nt  # Reservoir level decreases
    elif (Pd.iloc[j][i] - Pwr[j][i]) > Ptmax and (Pd.iloc[j][i] - Pwr[j][i]) <= (Ptmax + PDmin):  
                Pge[j][i] = PDmin  # Ensure minimum for diesel generator
                Pt[j][i] = (Pd.iloc[j][i] - Pwr[j][i]) - Pge[j][i]  # Hydraulic turbine complements everything
                Erj -= Pt[j][i] / nt  # Reservoir level decreases
    elif (Pd.iloc[j][i] - Pwr[j][i]) > (Ptmax + PDmin):  # Check if diesel and hydraulic can work together
                Pt[j][i] = Ptmax  # Maximum power of the turbine
                Pge[j][i] = Pd.iloc[j][i] - Pwr[j][i] - Pt[j][i]  # Diesel generator complements the rest
                Erj -= Ptmax / nt  # Reservoir level decreases


elif (Pd.iloc[j][i] - Pwr[j][i]) > Erj * nt:  # Check if reservoir has enough hydraulic energy to complement wind energy
        if Erj * nt > Ptmax and Erj * nt < (Ptmax + PDmin):
            if Ptmax >= PDmin and (Pd.iloc[j][i] - Pwr[j][i]) <= Ptmax:  # Ensure turbine stays below its power
                    Pge[j][i] = PDmin  # Ensure minimum for diesel generator
                    Pt[j][i] = (Pd.iloc[j][i] - Pwr[j][i]) - PDmin  # Hydraulic turbine complements everything
                    Erj -= Pt[j][i] / nt  # Reservoir level decreases
            else:
                    Pge[j][i] = Pd.iloc[j][i] - Pwr[j][i]  # Diesel generator produces everything since it can't produce below 120

        elif Erj * nt > (Ptmax + PDmin):  # Check if diesel and hydraulic can work together
                Pt[j][i] = Ptmax  # Maximum power of the turbine
                Pge[j][i] = Pd.iloc[j][i] - Pwr[j][i] - Pt[j][i]  # Diesel generator complements the rest
                Erj -= Ptmax / nt  # Reservoir level decreases
        else:
                Pge[j][i] = Pd.iloc[j][i] - Pwr[j][i]  # Only diesel ensures consumption is less than 120 kW

Er[j][i] = Erj  # Store the current value of reservoir energy temporarily in variable Erj





# pW = percentual de eólica na rede
pW = np.sum(Pwr) / np.sum(Pd.to_numpy()) * 100
print('Percentual de Energia eolica na rede = %3.2f' % pW)

# pH = percentual de energia hídrica na rede elétrica
pH = np.sum(Pt) / np.sum(Pd.to_numpy()) * 100
print('Percentual de Energia hidrica na rede = %3.2f' % pH)

# pER = percentual de penetração total de Renováveis (eólica+hidro)
pER = (np.sum(Pwr + Pt) / np.sum(Pd.to_numpy())) * 100
print('Total de RE = %3.2f' % pER)

# ERt = energia renovável aproveitada (eolica injetada diretamente e hidraulica), kWh
ERt = np.sum(Pwr + Pt)

# pDG = percentual de energia fossil na rede
pDG = np.sum(Pge) / np.sum(Pd.to_numpy()) * 100
print('Percentual de Energia fossil na rede = %3.2f' % pDG)

# pdiss = percentual de energia dissipada devido a reservatorio cheio
pdiss = np.sum(Erej) / (np.sum(Pwt) * ano) * 100
print('Percentual de  Energia dissipada devido a reservatorio cheio: %3.2f' % pdiss)






Percentual de Energia eolica na rede = nan
Percentual de Energia hidrica na rede = nan
Total de RE = nan
Percentual de Energia fossil na rede = nan
Percentual de  Energia dissipada devido a reservatorio cheio: 71.59


C:\Users\Gabriel\AppData\Local\Temp\ipykernel_1912\1406019611.py:116: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (Pd.iloc[j][i] - Pwr[j][i]) <= Ptmax:  # Check if the hydroturbine has enough power
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_1912\1406019611.py:119: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif (Pd.iloc[j][i] - Pwr[j][i]) > Ptmax and (Pd.iloc[j][i] - Pwr[j][i]) <= (Ptmax + PDmin):
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_1912\1406019611.py:123: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels